# FOUILLES DE DONNES MASSIVES

## NETTOYAGE

### Importation des données

In [67]:
import pandas as pd

In [68]:
pd.set_option('display.max_columns', 500)

df = pd.read_csv("guillaume.txt",sep=';')

C:\Users\vsigogneau\AppData\Local\Temp\ipykernel_3932\1736366370.py:3: DtypeWarning: Columns (1,2,5,6,7,8,9,15,16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("guillaume.txt",sep=';')


In [69]:
df.head()

,ZIBZIN,IDAvisAutorisationCheque,FlagImpaye,Montant,DateTransaction,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNb_RB,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr1,DiffDateTr2,DiffDateTr3,CA3TRetMtt,CA3TR,Heure
0,A013010004908126703060931,78643044,0,20,2017-02-01 07:32:14,1,0,0,0,551,0,0,0,"37,186667890919111","52,076033757361408",0,1,4,4,4,20,0,27134
1,A013011306908024927155000,78643045,0,20,2017-02-01 07:43:37,1,0,0,0,551,0,0,0,"48,844716275908937","52,076033757361408",1,2,"1,7976851851851852",4,4,"28,609999999999999","8,6099999999999994",27817
2,A013010002908283134592527,78643046,0,"57,640000000000001",2017-02-01 07:47:38,1,0,0,0,549,0,0,0,"73,118279569892479","52,076033757361408",0,1,4,4,4,"57,640000000000001",0,28058
3,A011010002908105209831316,78643047,0,"54,289999999999999",2017-02-01 07:48:48,0,1,1,1,267,0,0,0,"110,05692599620494","53,554233554497365",0,1,4,4,4,"54,289999999999999",0,28128
4,A013010041908000125652029,78643048,0,"26,899999999999999",2017-02-01 08:13:27,1,0,0,0,549,"3,769090654336556E-3","8,5863334286662614","1,1922637097134124E-3","45,36831264567185","52,076033757361408",1,1,"1,9971064814814814",4,4,"59,149999999999999","32,25",29607


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4646774 entries, 0 to 4646773
Data columns (total 23 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   ZIBZIN                    object
 1   IDAvisAutorisationCheque  object
 2   FlagImpaye                object
 3   Montant                   object
 4   DateTransaction           object
 5   CodeDecision              object
 6   VerifianceCPT1            object
 7   VerifianceCPT2            object
 8   VerifianceCPT3            object
 9   D2CB                      object
 10  ScoringFP1                object
 11  ScoringFP2                object
 12  ScoringFP3                object
 13  TauxImpNb_RB              object
 14  TauxImpNB_CPM             object
 15  EcartNumCheq              object
 16  NbrMagasin3J              object
 17  DiffDateTr1               object
 18  DiffDateTr2               object
 19  DiffDateTr3               object
 20  CA3TRetMtt                object
 21  CA3TR   

In [71]:
df['Montant'] = df['Montant'].str.replace(',','.') 
# Liste des colonnes à convertir en numérique
numeric_cols = ['VerifianceCPT1', 'VerifianceCPT2', 'VerifianceCPT3', 'D2CB', 'ScoringFP1',
                'ScoringFP2', 'ScoringFP3', 'TauxImpNb_RB', 'TauxImpNB_CPM', 'EcartNumCheq',
                'NbrMagasin3J', 'DiffDateTr1', 'DiffDateTr2', 'DiffDateTr3', 'CA3TRetMtt', 'CA3TR', 'Montant']

# Convertir les colonnes en types numériques
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')


In [72]:
# Créez de nouvelles colonnes pour le jour, le mois et l'année
df['DateTransaction'] = pd.to_datetime(df['DateTransaction'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Créez de nouvelles colonnes pour le jour, le mois et l'année, ainsi que l'heure, la minute et la seconde
df['Jour'] = df['DateTransaction'].dt.day
df['Jour'] = df['Jour'].fillna(0).astype(int)
df['Mois'] = df['DateTransaction'].dt.month
df['Mois'] = df['Mois'].fillna(0).astype(int)
df['Année'] = df['DateTransaction'].dt.year
df['Année'] = df['Année'].fillna(0).astype(int)
df['Heure2'] = df['DateTransaction'].dt.hour
df['Heure2'] = df['Heure2'].fillna(0).astype(int)
df['Minute'] = df['DateTransaction'].dt.minute
df['Minute'] = df['Minute'].fillna(0).astype(int)
df['Seconde'] = df['DateTransaction'].dt.second
df['Seconde'] = df['Seconde'].fillna(0).astype(int)

# Créez une nouvelle colonne 'Date' en combinant les composants jour, mois et année
df['Date'] = df['DateTransaction'].dt.date
df['Heures'] = pd.to_datetime(df[['Heure2', 'Minute', 'Seconde']].astype(str).agg(':'.join, axis=1), format='%H:%M:%S')
df['Heures'] = df['Heures'].dt.time

# Supprimez la colonne originale 'DateTransaction' si vous ne souhaitez plus la conserver
df = df.drop(['DateTransaction'], axis=1)

#Changement de type de colonne
df['CodeDecision'] = df['CodeDecision'].astype(str)
df['FlagImpaye'] = df['FlagImpaye'].astype(str)


In [73]:
# On supprime la ligne qui est étrange
index_flag_impaye = df[df['FlagImpaye'] == 'FlagImpaye'].index
# Supprimer les lignes correspondantes
df = df.drop(index_flag_impaye)

## RECHERCHE

In [74]:
date_debut = pd.to_datetime("2017-02-01").date()
date_fin = pd.to_datetime("2017-08-31").date()
df_train = df[(df['Date'] >= date_debut) & (df['Date'] <= date_fin)]


date_debut_test = pd.to_datetime("2017-09-01").date()
date_fin_test = pd.to_datetime("2017-11-30").date()
df_test = df[(df['Date'] >= date_debut_test) & (df['Date'] <= date_fin_test)]

In [83]:
X = df.drop('FlagImpaye', axis=1)
y = df['FlagImpaye']

X_train = df_train.drop('FlagImpaye', axis=1)
y_train = df_train['FlagImpaye']

X_test = df_test.drop('FlagImpaye', axis=1)
y_test = df_test['FlagImpaye']

In [84]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score

# Diviser les caractéristiques en numériques et catégorielles
numeric_features = X.select_dtypes(include=['int', 'float']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Construire le prétraitement des données
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Appliquer SMOTE sur les données d'entraînement
smote = SMOTE(random_state=42)
model = RandomForestClassifier(random_state=42)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('smote', smote),
                            ('classifier', model)])

pipeline.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = pipeline.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'SMOTE(random_state=42)' (type <class 'imblearn.over_sampling._smote.base.SMOTE'>) doesn't